In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import NarrowbandMetadata
from torchsig.signals.builders.constellation import ConstellationSignalBuilder
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
from torchsig.utils.dsp import (
    torchsig_complex_data_type,
    frequency_shift,
    multistage_polyphase_resampler
)

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 10, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal.

    """
    sample_rate = 10e6
    md = NarrowbandMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['qpsk'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ConstellationSignalBuilder(
        dataset_metadata = md, 
        class_name = 'qpsk',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale).astype(torchsig_complex_data_type)

    return signal

In [ ]:
def generate_tone_signal(num_iq_samples: int = 10, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband tone Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal.

    """
    sample_rate = 10e6
    md = NarrowbandMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['tone'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ToneSignalBuilder(
        dataset_metadata = md, 
        class_name = 'tone',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale).astype(torchsig_complex_data_type)

    return signal

In [ ]:
# test data
N = 4096
qpsk_bb_data = generate_qpsk_signal(num_iq_samples = N, scale = 1.0).data
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data

# upsample, rescale, then frequency shift
qpsk_8x_data = multistage_polyphase_resampler(qpsk_bb_data, 8.0)
tone_8x_data = multistage_polyphase_resampler(tone_bb_data, 8.0)

qpsk_data = frequency_shift(qpsk_8x_data, 0.125, 1.0)
qpsk_data = 2 * qpsk_data / np.max(np.abs(qpsk_data))

tone1 = frequency_shift(tone_8x_data, 0.10, 1.0)
tone1 = tone1 / np.max(np.abs(tone1)) 
tone2 = frequency_shift(tone_8x_data, 0.15, 1.0)
tone2 = tone2 / np.max(np.abs(tone2))
two_tone_data = 1 * (tone1 + tone2)

freq_vec = np.arange(-1.0/2,1.0/2,1.0/(N*8))

In [ ]:
# Table-based AM/AM, AM/PM Amplifier Model: examine Two Tone input response
auto_scale = False

# Define nonlinear amplifier model parameters
Pin  = 10**((np.array([-20., -10.,  0.,  5., 10. ])) / 10)
#Pin = np.arange(0.0, 10.0, 0.1)

Pout = 10**((np.array([-10.,   0.,  9., 9.9, 10. ])) / 10)
# Pout  = 10**((np.array([-25., -10.,  0.,  5., 10. ])) / 10)
# Pout = Pin
# Pout[-90:] = Pout[-90]

Phi  = np.deg2rad(np.array([-2., -4., 7., 12., 23.]))        
#Phi = np.zeros_like(Pin)

# test
data_in = two_tone_data
data_out = F.nonlinear_amplifier_table(
    data = data_in,
    Pin  = Pin,
    Pout = Pout,
    Phi  = Phi,
    auto_scale = auto_scale
)

input_mag = np.abs(data_in)
input_power = np.abs(data_in)**2
output_mag = np.abs(data_out)
output_power = np.abs(data_out)**2

input_phase_rad = np.angle(data_in)
output_phase_rad = np.angle(data_out)
phase_diff_rad = np.unwrap(output_phase_rad - input_phase_rad)
phase_diff_degree = np.rad2deg(phase_diff_rad)

### Plots
# AM/AM Power
fig = plt.figure(figsize=(8, 8))
plt.plot(Pin,Pout,'.-');
plt.plot(input_power,output_power,'k.');
plt.title('Nonlinear Amplifier Table Model AM/AM: Two Tone Test Signal',fontsize='large');
plt.xlabel('Pin (W)',fontsize='large');
plt.ylabel('Pout (W)',fontsize='large');
plt.legend(['Model','Test'],fontsize='large');
plt.show()

# AM/PM Phase
fig = plt.figure(figsize=(8, 8))
plt.plot(Pin,Phi,'.-');
plt.plot(input_power,phase_diff_rad,'k.');
plt.title('Nonlinear Amplifier Table Model AM/PM: Two Tone Test Signal',fontsize='large');
plt.xlabel('Pin (W)',fontsize='large');
plt.ylabel('Phi (rad)',fontsize='large');
plt.legend(['Model','Test'],fontsize='large');
plt.show()

# Spectrum
fig = plt.figure(figsize=(8, 8))
T = np.fft.fftshift(np.fft.fft(data_in))/(N*8);
T_nl = np.fft.fftshift(np.fft.fft(data_out))/(N*8);

plt.plot(freq_vec, 10*np.log10(np.abs(T_nl*T_nl)), 'b');
plt.plot(freq_vec, 10*np.log10(np.abs(T*T)), 'k-');
plt.ylim([-80, 10]);
plt.xlabel('Frequency (Fs norm)',fontsize='large');
plt.ylabel('Magnitude (log10)',fontsize='large');
plt.title('Table AM/AM AM/PM Amplifier Table Model Spectrum: Two Input Tones (0.1, 0.15)');
plt.legend(['Nonlinear','Linear'],fontsize='large', loc='upper left');
plt.show()

In [ ]:
# Table-based AM/AM, AM/PM Amplifier Model: examine QPSK signal input response
auto_scale = False

# nonlinear amplifier model parameters
Pin  = 10**((np.array([-20., -10.,  0.,  5., 10. ])) / 10)
#Pin = np.arange(0.0, 10.0, 0.1)

Pout = 10**((np.array([-10.,   0.,  9., 9.9, 10. ])) / 10)
# Pout  = 10**((np.array([-25., -10.,  0.,  5., 10. ])) / 10)
# Pout = Pin
# Pout[-90:] = Pout[-90]

Phi  = np.deg2rad(np.array([-2., -4., 7., 12., 23.]))        
#Phi = np.zeros_like(Pin)

# test
data_in = qpsk_data
data_out = F.nonlinear_amplifier_table(
    data = data_in,
    Pin  = Pin,
    Pout = Pout,
    Phi  = Phi,
    auto_scale = auto_scale    
)

input_mag = np.abs(data_in)
input_power = np.abs(data_in)**2
output_mag = np.abs(data_out)
output_power = np.abs(data_out)**2

input_phase_rad = np.angle(data_in)
output_phase_rad = np.angle(data_out)
phase_diff_rad = np.unwrap(output_phase_rad - input_phase_rad)
phase_diff_degree = np.rad2deg(phase_diff_rad)

### Plots
# AM/AM Power
fig = plt.figure(figsize=(8, 8))
plt.plot(Pin,Pout,'.-');
plt.plot(input_power,output_power,'k.');
plt.title('Nonlinear Amplifier Table Model AM/AM: QPSK Test Signal',fontsize='large');
plt.xlabel('Pin (W)',fontsize='large');
plt.ylabel('Pout (W)',fontsize='large');
plt.legend(['Model','Test'],fontsize='large');
plt.show()

# AM/PM Phase
fig = plt.figure(figsize=(8, 8))
plt.plot(Pin,Phi,'.-');
plt.plot(input_power,phase_diff_rad,'k.');
plt.title('Nonlinear Amplifier Table Model AM/PM: QPSK Test Signal',fontsize='large');
plt.xlabel('Pin (W)',fontsize='large');
plt.ylabel('Phi (rad)',fontsize='large');
plt.legend(['Model','Test'],fontsize='large');
plt.show()

# Spectrum
fig = plt.figure(figsize=(8, 8))
T = np.fft.fftshift(np.fft.fft(data_in))/(N*8);
T_nl = np.fft.fftshift(np.fft.fft(data_out))/(N*8);

plt.plot(freq_vec, 10*np.log10(np.abs(T_nl*T_nl)), 'b');
plt.plot(freq_vec, 10*np.log10(np.abs(T*T)), 'k-');
plt.ylim([-80, 10]);
plt.xlabel('Frequency (Fs norm)',fontsize='large');
plt.ylabel('Magnitude (log10)',fontsize='large');
plt.title('Nonlinear Amplifier Table Model AM/PM Spectrum: QPSK');
plt.legend(['Nonlinear','Linear'],fontsize='large', loc='upper left');
plt.show()

In [ ]:
# Function-based AM/AM, AM/PM Amplifier Model: examine Two Tone input response
auto_scale = False

# Define nonlinear amplifier model parameters
gain = 8.0
psat_backoff = 5.0
phi_rad = 0.2

# test
data_in = two_tone_data
data_out = F.nonlinear_amplifier(
    data = data_in,
    gain  = gain,
    psat_backoff = psat_backoff,
    phi_rad  = phi_rad,
    auto_scale = auto_scale
)

input_mag = np.abs(data_in)
input_power = np.abs(data_in)**2
output_mag = np.abs(data_out)
output_power = np.abs(data_out)**2

input_phase_rad = np.angle(data_in)
output_phase_rad = np.angle(data_out)
phase_diff_rad = np.unwrap(output_phase_rad - input_phase_rad)
phase_diff_degree = np.rad2deg(phase_diff_rad)

### Plots

# ideal linear gain response
linear_input_power = np.linspace(0.0, np.max(input_power), 1000)
linear_output_power = gain * linear_input_power

# AM/AM Power
fig = plt.figure(figsize=(8, 8))
plt.plot(linear_input_power,linear_output_power,'b--');
plt.plot(input_power,output_power,'k.');
plt.title('Nonlinear Amplifier Model AM/AM: Two Tone Test Signal',fontsize='large');
plt.xlabel('Pin (W)',fontsize='large');
plt.ylabel('Pout (W)',fontsize='large');
plt.legend(['Linear','Test'],fontsize='large');
plt.show()

# AM/PM Phase
fig = plt.figure(figsize=(8, 8))
plt.plot(input_power,phase_diff_rad,'k.');
plt.title('Nonlinear Amplifier Model AM/PM: Two Tone Test Signal',fontsize='large');
plt.xlabel('Pin (W)',fontsize='large');
plt.ylabel('Phi (rad)',fontsize='large');
plt.legend(['Model','Test'],fontsize='large');
plt.show()

# Spectrum
fig = plt.figure(figsize=(8, 8))
T = np.fft.fftshift(np.fft.fft(data_in))/(N*8);
T_nl = np.fft.fftshift(np.fft.fft(data_out))/(N*8);

plt.plot(freq_vec, 10*np.log10(np.abs(T_nl*T_nl)), 'b');
plt.plot(freq_vec, 10*np.log10(np.abs(T*T)), 'k-');
plt.ylim([-80, 10]);
plt.xlabel('Frequency (Fs norm)',fontsize='large');
plt.ylabel('Magnitude (log10)',fontsize='large');
plt.title('Nonlinear Amplifier Model Spectrum: Two Input Tones (0.1, 0.15)');
plt.legend(['Nonlinear','Linear'],fontsize='large', loc='upper left');
plt.show()

In [ ]:
# Function-based AM/AM, AM/PM Amplifier Model: examine QPSK signal input response
auto_scale = False

# Define nonlinear amplifier model parameters
gain = 8.0
psat_backoff = 5.0
phi_rad = 0.2

# test
data_in = two_tone_data
data_out = F.nonlinear_amplifier(
    data = data_in,
    gain  = gain,
    psat_backoff = psat_backoff,
    phi_rad  = phi_rad,
    auto_scale = auto_scale
)

input_mag = np.abs(data_in)
input_power = np.abs(data_in)**2
output_mag = np.abs(data_out)
output_power = np.abs(data_out)**2

input_phase_rad = np.angle(data_in)
output_phase_rad = np.angle(data_out)
phase_diff_rad = np.unwrap(output_phase_rad - input_phase_rad)
phase_diff_degree = np.rad2deg(phase_diff_rad)

### Plots

# ideal linear gain response
linear_input_power = np.linspace(0.0, np.max(input_power), 1000)
linear_output_power = gain * linear_input_power

# AM/AM Power
fig = plt.figure(figsize=(8, 8))
plt.plot(linear_input_power,linear_output_power,'b--');
plt.plot(input_power,output_power,'k.');
plt.title('Nonlinear Amplifier Model AM/AM: Two Tone Test Signal',fontsize='large');
plt.xlabel('Pin (W)',fontsize='large');
plt.ylabel('Pout (W)',fontsize='large');
plt.legend(['Linear','Test'],fontsize='large');
plt.show()

# AM/PM Phase
fig = plt.figure(figsize=(8, 8))
plt.plot(input_power,phase_diff_rad,'k.');
plt.title('Nonlinear Amplifier Model AM/PM: Two Tone Test Signal',fontsize='large');
plt.xlabel('Pin (W)',fontsize='large');
plt.ylabel('Phi (rad)',fontsize='large');
plt.legend(['Model','Test'],fontsize='large');
plt.show()

# Spectrum
fig = plt.figure(figsize=(8, 8))
T = np.fft.fftshift(np.fft.fft(data_in))/(N*8);
T_nl = np.fft.fftshift(np.fft.fft(data_out))/(N*8);

plt.plot(freq_vec, 10*np.log10(np.abs(T_nl*T_nl)), 'b');
plt.plot(freq_vec, 10*np.log10(np.abs(T*T)), 'k-');
plt.ylim([-80, 10]);
plt.xlabel('Frequency (Fs norm)',fontsize='large');
plt.ylabel('Magnitude (log10)',fontsize='large');
plt.title('Nonlinear Amplifier Model Spectrum: Two Input Tones (0.1, 0.15)');
plt.legend(['Nonlinear','Linear'],fontsize='large', loc='upper left');
plt.show()

In [ ]:
# Automatic output scaling example

# Define nonlinear amplifier model parameters
gain = 8.0
psat_backoff = 5.0
phi_rad = 0.2

# test
data_in = qpsk_data
unscaled_data_out = F.nonlinear_amplifier(
    data = data_in,
    gain  = gain,
    psat_backoff = psat_backoff,
    phi_rad  = phi_rad,
    auto_scale = False
)
autoscaled_data_out = F.nonlinear_amplifier(
    data = data_in,
    gain  = gain,
    psat_backoff = psat_backoff,
    phi_rad  = phi_rad,
    auto_scale = True
)

input_mag = np.abs(data_in)
input_power = np.abs(data_in)**2

unscaled_output_mag = np.abs(unscaled_data_out)
unscaled_output_power = np.abs(unscaled_data_out)**2
autoscaled_output_mag = np.abs(autoscaled_data_out)
autoscaled_output_power = np.abs(autoscaled_data_out)**2

### Plots

# ideal linear gain response
linear_input_power = np.linspace(0.0, np.max(input_power), 1000)
linear_output_power = gain * linear_input_power

# AM/AM Power
fig = plt.figure(figsize=(8, 8))
plt.plot(linear_input_power,linear_output_power,'b--');
plt.plot(input_power,unscaled_output_power,'g.');
plt.plot(input_power,autoscaled_output_power,'k.');
plt.title('Nonlinear Amplifier: Autoscaling',fontsize='large');
plt.xlabel('Pin (W)',fontsize='large');
plt.ylabel('Pout (W)',fontsize='large');
plt.legend(['Linear','Unscaled','Autoscaled'],fontsize='large');
plt.show()

# Spectrum
fig = plt.figure(figsize=(8, 8))
T = np.fft.fftshift(np.fft.fft(data_in))/(N*8);
T_nl = np.fft.fftshift(np.fft.fft(autoscaled_data_out))/(N*8);

plt.plot(freq_vec, 10*np.log10(np.abs(T_nl*T_nl)), 'b');
plt.plot(freq_vec, 10*np.log10(np.abs(T*T)), 'k-');
plt.ylim([-80, 10]);
plt.xlabel('Frequency (Fs norm)',fontsize='large');
plt.ylabel('Magnitude (log10)',fontsize='large');
plt.title('Nonlinear Amplifier Model Spectrum: QPSK (Autoscaled)');
plt.legend(['Autoscaled','Linear'],fontsize='large', loc='upper left');
plt.show()

In [ ]:
# Function-based approximation model characterization

def nonlinear_amplifier_proto(input_power, psat, linear_gain=1.0, function_type='tanh'):
    """Model a nonlinear amplifier that saturates at a specified power level (Psat).
    
    Args:
        input_power (float or ndarray): Input power values
        psat (float): Saturation output power
        linear_gain (float): Linear gain in the small-signal region
        function_type (str): Type of saturation function: 'tanh' or 'sigmoid'
    
    Returns:
        float or ndarray: Output power values
    """
    # Calculate the scale factor to achieve the desired linear gain
    scale_factor = psat / linear_gain
    
    if function_type.lower() == 'tanh':
        # Hyperbolic tangent implementation
        # Passes through (0,0) and asymptotically approaches psat
        output_power = psat * np.tanh(input_power / scale_factor)
    
    elif function_type.lower() == 'sigmoid':
        # Sigmoid implementation
        # Modified to pass through (0,0) and saturate at psat
        x = input_power / (scale_factor * 0.5)  # Scale factor adjustment for sigmoid
        output_power = psat * (2 * (1 / (1 + np.exp(-x))) - 1)
        
    else:
        raise ValueError("function_type must be either 'tanh' or 'sigmoid'")
    
    return output_power

def find_p1db(input_power, output_power, linear_response):
    """
    Find the 1dB compression point (P1dB) in the amplifier response.
    
    Args:
        input_power (ndarray): Input power values
        output_power (ndarray): Actual output power values
        linear_response (ndarray): Ideal linear output power values
    
    Returns:
        tuple: Input power and output power at the P1dB point
    """
    # Calculate compression in dB
    with np.errstate(divide='ignore', invalid='ignore'):
        compression_db = 10 * np.log10(linear_response / output_power)
    
    # Find valid indices where compression is positive but not too large
    valid_indices = (compression_db > 0) & (compression_db < 10) & np.isfinite(compression_db)
    
    if np.any(valid_indices):
        valid_indices_array = np.where(valid_indices)[0]
        p1db_index = valid_indices_array[np.argmin(np.abs(compression_db[valid_indices] - 1.0))]
        return input_power[p1db_index], output_power[p1db_index]
    else:
        return None, None


# Define amplifier parameters
psat = 10.0        # Saturation power level
linear_gain = 1.0  # Linear gain in the small-signal region

# Generate input power range
input_power = np.linspace(0, 3*psat/linear_gain, 1000)

# Calculate amplifier response with both functions
output_tanh = nonlinear_amplifier_proto(input_power, psat, linear_gain, function_type='tanh')
output_sigmoid = nonlinear_amplifier_proto(input_power, psat, linear_gain, function_type='sigmoid')

# Calculate linear response for comparison
linear_response = linear_gain * input_power

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(input_power, output_tanh, 'b--', linewidth=2, label='Tanh Model')
plt.plot(input_power, output_sigmoid, 'g--', linewidth=2, label='Sigmoid Model')
plt.plot(input_power, linear_response, 'r--', linewidth=1.5, label='Linear Response')
plt.axhline(y=psat, color='k', linestyle='-.', linewidth=1.5, label='Psat')

# Find P1dB points for each function
p1db_in_tanh, p1db_out_tanh = find_p1db(input_power, output_tanh, linear_response)
p1db_in_sigmoid, p1db_out_sigmoid = find_p1db(input_power, output_sigmoid, linear_response)

# Plot P1dB points and calculate distance to Psat
if p1db_in_tanh is not None:
    plt.plot(p1db_in_tanh, p1db_out_tanh, 'bo', markersize=8, label='P1dB (Tanh)')
    p1db_to_psat_db_tanh = 10 * np.log10(psat / p1db_out_tanh)
    # print(f"P1dB point (Tanh): Input = {p1db_in_tanh:.3f}, Output = {p1db_out_tanh:.3f}")
    # print(f"Distance from P1dB to Psat (Tanh): {p1db_to_psat_db_tanh:.2f} dB")

if p1db_in_sigmoid is not None:
    plt.plot(p1db_in_sigmoid, p1db_out_sigmoid, 'go', markersize=8, label='P1dB (Sigmoid)')
    p1db_to_psat_db_sigmoid = 10 * np.log10(psat / p1db_out_sigmoid)
    # print(f"P1dB point (Sigmoid): Input = {p1db_in_sigmoid:.3f}, Output = {p1db_out_sigmoid:.3f}")
    # print(f"Distance from P1dB to Psat (Sigmoid): {p1db_to_psat_db_sigmoid:.2f} dB")

# Additional plot settings
plt.xlabel('Input Power (W)')
plt.ylabel('Output Power (W)')
plt.title('Nonlinear Amplifier Model (Psat Saturation)')
plt.grid(True)
plt.legend()
plt.show()

# Compare different saturation rates with tanh model
plt.figure(figsize=(10, 6))

# Calculate responses with different saturation rates
output_default = nonlinear_amplifier_proto(input_power, psat, linear_gain=1.0)
output_fast = nonlinear_amplifier_proto(input_power, psat, linear_gain=10.0)
output_slow = nonlinear_amplifier_proto(input_power, psat, linear_gain=0.5)

plt.plot(input_power, linear_response, 'r--', linewidth=1.5, label='Linear Gain 1.0')
plt.plot(input_power, output_default, 'b-', linewidth=2, label='linear_gain=1.0')
plt.plot(input_power, output_fast, 'g-', linewidth=2, label='linear_gain=10.0')
plt.plot(input_power, output_slow, 'm-', linewidth=2, label='linear_gain=0.5')
plt.axhline(y=psat, color='k', linestyle='-.', linewidth=1.5, label='Psat')

plt.xlabel('Input Power (W)')
plt.ylabel('Output Power (W)')
plt.title('Effect of Linear Gain on Amplifier Response')
plt.grid(True)
plt.legend()
plt.show()